In [2]:
import pandas as pd
import numpy as np
from Lib import Algen_lib as lib

# Load Data

In [4]:
#Data Latih
df = pd.read_excel('data/sms_clean_norm_+_new_data_3.xlsx')

corpusA = df.sms.tolist()
labelA = df.label.tolist()
len(df)

1143

# Ekstraksi Fitur

In [5]:
fitur = lib.feature_extraction(corpusA)

# Seleksi Fitur Menggunaka Algoritma Genetika

In [13]:
jumlah_populasi = 50
jumlah_point = 1000 #jumlah titik point crossover
prob_mutasi = 0.2 #probabilitas permutasi

K=30

metode = "tfidf"

alpha = 1
target = 0.99 #target akurasi

panjang_fitur = len(fitur)

features_1_bin = [1 for i in range(panjang_fitur)]
pop = lib.create_population(jumlah_populasi-1, panjang_fitur)
pop.append(features_1_bin)

fns_1 = lib.fitness_kf(corpusA, labelA, features_1_bin, fitur, alpha = alpha, metode = metode, K=K)
print("fitness all features:",fns_1)

fitnes_pop = list()
# temp_features_fitness = list()
for features_bin in pop:
    fns = lib.fitness_kf(corpusA, labelA, features_bin, fitur, alpha = alpha, metode = metode, K=K)
    fitnes_pop.append(fns)
#     temp_features_fitness.append([fns, features_bin])
# fitnes_pop.append(fns_1)

print(len(fitnes_pop),"|",len(pop))

pop_used = dict()
popp_0 = list(pop)
for p, fit in zip(popp_0, fitnes_pop):
    bin_str = lib.int_to_str(p) 
    pop_used.update({bin_str:fit})

# print("")
    
# print(pop[0])

best_fitur_list = list()
best_fitness_list = list()
best_generasi_list = list()

best_jumlah_fitur_list = list()

# save_aja = list()

gen=0
while True:
    gen+=1
    print("-"*25)
    print("Generasi ke",gen , "[Terbaik]")
    rw = lib.get_roulette_wheel_(fitnes_pop, inc=jumlah_populasi)
    parents = lib.select_parents(rw)

    best_fitur_list_per_generasi = list()
    best_fitness_list_per_generasi = list()
    for p in parents:
        best_generasi_list.append(gen)
        family_fitness=list()
        family_fitur=list()

        mama_index = p[0]
        papa_index = p[1]
        
        m4ms = fitnes_pop[mama_index]
        family_fitness.append(m4ms)
        m4mr = list(pop[mama_index])
        family_fitur.append(m4mr)
#         best_jumlah_fitur_list.append(sum(m4mr))
        
        p4ps = fitnes_pop[papa_index]
        family_fitness.append(p4ps)
        p4pr = list(pop[papa_index])
        family_fitur.append(p4pr)
#         best_jumlah_fitur_list.append(sum(p4pr))
        
        #crossover dan mutasi
        bin_mama = list(pop[mama_index])
        bin_papa = list(pop[papa_index])
        anak_binary = lib.crossover(bin_mama, bin_papa, panjang_fitur,  jumlah_point = jumlah_point, prob_mutasi = prob_mutasi)
        
        #mencari fitness untuk child
        a1 = anak_binary[0]
        a2 = anak_binary[1]
        a3 = anak_binary[2]
#         print(a2)
        
        if sum(a1)<=0:
            a1[-1]=1
            a1[-2]=1
        if sum(a2)<=0:
            a2[-1]=1
            a2[-2]=1
        if sum(a3)<=0:
            a3[-1]=1
            a3[-2]=1

        str_a1 = lib.int_to_str(a1)
        if str_a1 in pop_used:
            family_fitur.append(a1)
            family_fitness.append(pop_used[str_a1])
#             best_jumlah_fitur_list.append(sum(a1))
            
        else:
            family_fitur.append(a1)
            fitness_anak_1 = lib.fitness_kf(corpusA, labelA, a1, fitur, alpha = alpha, metode = metode, K=K)
            family_fitness.append(fitness_anak_1)
            pop_used.update({str_a1:fitness_anak_1})
#             best_jumlah_fitur_list.append(sum(a1))
            
        str_a2 = lib.int_to_str(a2)
        if str_a2 in pop_used:
            family_fitur.append(a2)
            family_fitness.append(pop_used[str_a2])
#             best_jumlah_fitur_list.append(sum(a2))
        else:
            family_fitur.append(a2)
            fitness_anak_2 = lib.fitness_kf(corpusA, labelA, a2, fitur, alpha = alpha, metode = metode, K=K)
            family_fitness.append(fitness_anak_2)
            pop_used.update({str_a2:fitness_anak_2})
#             best_jumlah_fitur_list.append(sum(a2))
            
        str_a3 = lib.int_to_str(a3)
        if str_a3 in pop_used:
            family_fitur.append(a3)
            family_fitness.append(pop_used[str_a3])
#             best_jumlah_fitur_list.append(sum(a3))
        else:
            family_fitur.append(a3)
            fitness_anak_3 = lib.fitness_kf(corpusA, labelA, a3, fitur, alpha = alpha, metode = metode, K=K)
            family_fitness.append(fitness_anak_3)
            pop_used.update({str_a3:fitness_anak_3})
#             best_jumlah_fitur_list.append(sum(a3))

        
        #mencari fitness terbaik untuk satu keluarga
        best_family_fitness = max(family_fitness)
        bf_index = family_fitness.index(best_family_fitness)
        best_family_fitur = family_fitur[bf_index]

        best_fitness_list.append(best_family_fitness)
        best_fitur_list.append(best_family_fitur)

        best_fitness_list_per_generasi.append(best_family_fitness)
        best_fitur_list_per_generasi.append(best_family_fitur)
        best_jumlah_fitur_list.append(sum(best_family_fitur))

        print("   ",best_family_fitness)
        print("   ","*"*20,sum(best_family_fitur),"/",len(best_family_fitur),"->",panjang_fitur,"|",gen)

    pop = list(best_fitur_list_per_generasi)
#     pop.append(features_1_bin)
    fitnes_pop = list(best_fitness_list_per_generasi)
#     fitnes_pop.append(fns_1)

    if len(pop)<2 or max(best_fitness_list)>=target:
        print("="*70)
        print("Best",max(best_fitness_list))
#         print("Generasi ke-",best_generasi_list[best_fitness_list.index(max(best_fitness_list))])
        good_fitur = best_fitur_list[best_fitness_list.index(max(best_fitness_list))]
        best_fitness_list.index(max(best_fitness_list))
        print('jumlah fitur     ', sum(good_fitur))
        print('jumlah fitur asli', panjang_fitur)
        print('good_fitur')
        break

fitness all features: 0.9353126405757984
50 | 50
-------------------------
Generasi ke 1 [Terbaik]
    0.920400359874044
    ******************** 3284 / 4378 -> 4378 | 1
    0.926563202878992
    ******************** 3309 / 4378 -> 4378 | 1
    0.9212550607287451
    ******************** 2227 / 4378 -> 4378 | 1
    0.9195006747638323
    ******************** 3261 / 4378 -> 4378 | 1
    0.9247863247863246
    ******************** 3270 / 4378 -> 4378 | 1
    0.926540710751237
    ******************** 3272 / 4378 -> 4378 | 1
    0.9273504273504272
    ******************** 3303 / 4378 -> 4378 | 1
    0.9230319388214123
    ******************** 3247 / 4378 -> 4378 | 1
    0.923009446693657
    ******************** 3289 / 4378 -> 4378 | 1
    0.9335357624831305
    ******************** 3296 / 4378 -> 4378 | 1
    0.9273729194781825
    ******************** 3286 / 4378 -> 4378 | 1
    0.9220872694556902
    ******************** 2191 / 4378 -> 4378 | 1
    0.9273729194781823
    **************

In [44]:
#Testing
hasil_seleksi_fitur = good_fitur
lib.fitness_kf(corpusA, labelA, hasil_seleksi_fitur, fitur, alpha = alpha, metode = metode, K=K)

0.9413944343617666

In [25]:
for i in np.array(fitur)[lib.get_index(hasil_seleksi_fitur)]:
    print(i, end=", ")

0, 000, 000untuk, 001, 0016282190254447, 0016282333306308, 0016282336033777, 0016285286552555, 001948, 002218, 0035, 0065, 0074, 01, 0100, 0101, 01016, 01123190427523627, 012786, 018801, 02, 021, 02129657333, 02136146799, 02152971540, 0215649770, 0215891174, 02199545139, 0226, 0231327xxx, 024, 02599875, 02749447574, 0277285682, 0282566132, 03, 0385, 04, 0544, 06, 076, 08, 0809, 081215008228, 081218844557, 081218986001, 081219550857, 081223052854, 081281108799, 0812xxxxxxx, 081310537888, 081310xx, 081316005639, 081316523061, 081316899123, 081330134449, 081356256931, 081389909777, 081571xxx, 081584650877, 081586769255, 0815xxxxxxx, 0816288602, 0818433247, 0818643929, 081931753240, 0819882923, 082, 0821, 082110001021, 082122483312, 082127377377, 082133271400, 082189983017, 082193609797, 0823, 082310204777, 082319733334, 082325557768, 082327327705, 082328823345, 082329676767, 082374432257, 082374910777, 083139195872, 085104894998, 0852, 085215113xxx, 085216181177, 085217845777, 08521798429

jtuh, juke, juli, jumat, jumlah, jun, juni, junior, juplo, jurus, jusman, juta, jxpc42, k, ka, kabar, kabarin, kabel, kacamata, kadang, kadisdik, kado, kagi, kait, kak, kaka, kaldaluarsa, kalem, kali, kalimat, kalo, kamar, kamera, kamis, kampuang, kampung, kampunggajah, kampus, kamrin, kamu, kan, kana, kang, kantin, kanto, kantor, kaprodi, karakter, karet, karim, karna, kartu, kartuhalo, karyawan, kas, kasih, kasur, kata, katalog, kategori, kav, kav50, kawal, kayak, kbtulan, kdg, kdgkan, ke10, ke123, ke2, ke4, kebaperan, kebebasa, keburu, kebut, kecil, kedlmn, keik, keisat, kejar, kejut, kejutanindosat, kejutantriindonesia, kejutanundiantri, kelik, keliling, kelompok, keluar, keluarga, kemarin, kembaliin, kembang, ken, kena, kenal, kenan, kenyang, kep, keren, kering, kerja, kesana, kesini, ketemu, ketik, kewirausahaan, keyboard, keyword, kfc, khilaf, kholik, khusus, ki, kikuk, kilat, kini, kios, kira, kira2, kirain, kirim, kirimin, kisar, kiwi, kjtn, kk, kke, klasifikasi, klau, klik, k

, tselthr, tst88, tsu59, ttd, ttg, tu, tua, tuanai, tugas, tugu, tuju, tuk, tukokno, tulang2an, tulis, tumben, tunggal, tunggu, turbo, turun, tutup, tv, tw96d, twitter, type, u, uang, uas, uasnya, ubah, uber, udah, udh, udunan, ujannya, ujiklyakankerjaberstifikat, ujug2, ulanag, ulg2, ulsa, ultima, umum, umur, unbearable, undang, undi, undian3, undianbri, undianindofood, undianmtronik, undur, unisba, unit, unlimited, unpad, unruk, untk, untungbeliung, up, upa, update, upi, upinet, upnyaa, urang, urangbandung, urg, urgent, urus, us6, usa, usah, usahain, use, ustad, usul, utama, utk, uts, v, v10, v20, v25, v5, v712, value, values, vape, vas280, vchr, verifikasi, versi, very, via, view, vii, vip, vipgames, visa, volume, voucher, vr25d, vu, wa, waaah, waalaikumsalam, waalaikumsalamin, wacana, wah, wahyu, wahyudi, wahyudin, wajah, wajib, wakil, waktu, waktupenguji, walfaidin, wallet, wallppr, walopun, waminkum, war, wardoyo, warkop, warna, warteg, wassalamu, wb, webnode, webs, website, webs

In [26]:
fitur_pilihan = np.array(fitur)[lib.get_index(good_fitur)]
print(len(fitur_pilihan))
fitur_pilihan

3406


array(['0', '000', '000untuk', ..., 'yunit', 'zalora', 'zarkasi'],
      dtype='<U26')

In [8]:

dict_data = {
    "fitness":best_fitness_list,
    "jumlah_fitur":best_jumlah_fitur_list,
    "generasi":best_generasi_list,
}

df_hasil = pd.DataFrame.from_dict(dict_data)
df_hasil


ValueError: arrays must all be same length

In [11]:
print(len(best_fitness_list))

52


In [43]:
cek = 0.98

grouped = df_hasil.groupby('generasi')
grouped.filter(lambda x: x['fitness'].mean() > cek)

,fitness,generasi
